In [0]:
%pip install openpyxl

import pandas as pd
import numpy as np

In [0]:
responses_path = r'/Volumes/prd_mega/sdgreg25/vdgreg25/Documents/MA Survey/MA Survey Results.csv'

responses_df_raw = pd.read_csv(responses_path)

survey_path = r'/Volumes/prd_mega/sdgreg25/vdgreg25/Documents/temp/MA_Survey_Final.xlsx'

choice_df_raw = pd.read_excel(survey_path, sheet_name='choices', engine='openpyxl')

question_df_raw = pd.read_excel(survey_path, sheet_name='survey', engine='openpyxl')

In [0]:
qdf = question_df_raw[145:][question_df_raw['type'].str.contains('select') | question_df_raw['type'].str.contains('text')]

cdf = choice_df_raw.copy()

rdf = responses_df_raw.copy()

display(rdf)

In [0]:
qdf

In [0]:
# score dataframe
sdf = cdf.copy()

sections = sdf['list_name'].drop_duplicates().to_list()

for s in sections:
    idcs = sdf[sdf['list_name'] == s].index
    j = 0
    for i in idcs:
        sdf.loc[i, 'score'] = j
        j += 1

display(sdf)




In [0]:
sections

## Data System Score

In [0]:


mas = rdf.loc[:,['ma_name','ms_name','caseid']]

# Average for the MIS and ecohesion systems
answer_scores = {
 'meeting_needs': {'fully':2,
    'partially':1,
    'minimally':0,
    },
 
 'updates':{'yes_maintenance':3,
    'yes_maintenance_and_improvements':2,
    'no_but_adds_regulation':1,
    'not_at_all':0
    },
 
 'api_data_transmission':{
        'yes':2,
        'no':0,
        'no_but_pursuing':1,
        'does_not_know':0,
    },
 
 'barriers_to_api_implementation':{'lack_of_technical_capacity':-1,
    'budget_constraints':-1,
    'regulatory_restrictions':-1,
    'legacy_issues':-1,
    'security_concerns':-1,
    'satisfied':-1,
    'other':-1,
    },
 }

# boost score for having an answer, assuming this means more connections
text_boosts = ['regional_national_integration_existing',
 'eu_integration_existing',
 'regional_national_integration_wants',
 'eu_integration_wants']

# assume this means more impediments
text_impediments=['other_api_barriers']

def score_systems(row):
    caseid = row['caseid']
    answer_row = rdf[rdf['caseid'] == caseid].squeeze()

    all_questions = qdf['name'].tolist()

    system_questions = [q for q in all_questions if 'mis_' in q.lower() or 'e_cohesion_' in q.lower()]

    score = 0
    total_questions = 0
    for q in system_questions:
        
        base_questions = q.replace('mis_financial_','').replace('mis_me_','').replace('mis_', '').replace('e_cohesion_', '')
        base_questions = base_questions.split(' ')

        for base_question in base_questions:
            if not pd.isna(answer_row[q]):
                if base_question in answer_scores.keys():
                    answers = answer_row[q].split(' ')
                    for answer in answers:
                        total_questions += 1
                        to_add = answer_scores[base_question][answer]
                        score += to_add
                elif base_question in text_boosts:
                    total_questions += 1
                    score = score + 1
                elif base_question in text_impediments:
                    total_questions += 1
                    score = score - 1
                
    return [score, total_questions]

mas['system_score & total_questions'] = mas.apply(lambda row: score_systems(row), axis=1)

mas['total_system_score'] = mas['system_score & total_questions'].apply(lambda x: x[0])
mas['total_questions'] = mas['system_score & total_questions'].apply(lambda x: x[1])

mas['system_score'] = mas['total_system_score'] / mas['total_questions']

display(mas)

In [0]:
import pandas as pd

# Sort by system_score descending
mas_sorted = mas.sort_values(by='system_score', ascending=False)

# Define your top percentile (e.g., 10%)
percentile = 0.10  # top 10%

# Calculate how many rows that is
top_n = int(len(mas_sorted) * percentile)

# Get the top X percentile rows
top_df = mas_sorted.head(top_n)

top_df